In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# URL to the CSV file
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"

# Load the data
df = pd.read_csv(URL)
print('Data downloaded and read into a dataframe!')

# Initialize the Dash app
app = dash.Dash(__name__)

year_list = [i for i in range(1980, 2024)]

app.layout = html.Div(children=[
    html.H1("Automobile Sales Statistics Dashboard",
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}),
    
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value="Select Statistics",
        style={'width': '60%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    dcc.Dropdown(
        id="select-year",
        options=[{'label': str(i), 'value': i} for i in year_list],
        placeholder="Select year",
        value="Select year",
        style={'width': '60%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    html.Div([
        html.Div(id="output-container", className="chart-grid", )
    ])
])

# Disable year selection based on selected statistics
@app.callback(
    Output(component_id="select-year", component_property="disabled"),
    Input(component_id="dropdown-statistics", component_property="value")
)
def update_input_container(selected_statistics):
    return False if selected_statistics == 'Yearly Statistics' else True

# Update output container based on selected statistics and year
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')]
)
def update_output_container(selected_statistics, selected_year):
    if selected_statistics == 'Recession Period Statistics':
        recession_data = df[df['Recession'] == 1]
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(figure=px.line(yearly_rec, x='Year', y='Automobile_Sales', title="ANNUAL AUTOMOBILE SALES During Recession"))
       
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(figure=px.bar(average_sales, x="Vehicle_Type", y="Automobile_Sales", title="Vehicle_Type VS Sales"))
        
        exp_rec = recession_data.groupby('Vehicle_Type')["Advertising_Expenditure"].sum().reset_index()
        R_chart3 = dcc.Graph(figure=px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type', title="Advertisement Expenditure for Different Vehicles"))
        
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type', labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'}, title='Effect of Unemployment Rate on Sales of Diff Vehicles'))
        
        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)], style={'display': 'flex'})
        ]
    
    elif selected_year and selected_statistics == 'Yearly Statistics':
        yearly_data = df[df['Year'] == int(selected_year)]
        yas = yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales', title="ANNUAL AUTOMOBILE SALES"))
        
        mas = yearly_data.groupby('Month')['Automobile_Sales'].mean().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas, x="Month", y="Automobile_Sales", title="MONTHLY AUTOMOBILE SALES"))
        
        avr_vdata = yearly_data.groupby('Vehicle_Type')["Automobile_Sales"].mean().reset_index()
        Y_chart3 = dcc.Graph(figure=px.bar(avr_vdata, x="Vehicle_Type", y="Automobile_Sales", title='Average Vehicles Sold by Vehicle Type in the year {}'.format(selected_year)))
        
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data, values='Advertising_Expenditure', names="Vehicle_Type", title='Total Advertisement Expenditure for Each Vehicle'))
        
        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)], style={'display': 'flex'})
        ]

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\ps289\AppData\Local\Temp\ipykernel_23560\1995523463.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\ps289\AppData\Local\Temp\ipykernel_23560\1995523463.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Data downloaded and read into a dataframe!
